dendrogram 

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import AggglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import make_blobs

X,_=make_blobs(n_samples=30,centers=3,random_state=42,cluster_std=10)
clustering=AgglomerativeClustering(n_clusters=3)
labelsd=clustering.fit_predict(X)

ImportError: cannot import name 'AggglomerativeClustering' from 'sklearn.cluster' (/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/cluster/__init__.py)